# API Server

In [1]:
# !pip install ninja opensimplex gradio moviepy==1.0.3 tts pyrebase
# !apt install ffmpeg
# !pip install git+https://github.com/1adrianb/face-alignment@v1.0.1
# !pip install fbpca boto3 requests==2.23.0 #urllib3==1.25.11
# !git submodule update --init --recursive

# !python -c "import nltk; nltk.download('wordnet')" 

In [2]:
import os

# 添加代码
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
# """
# 1、os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"#按照PCI_BUS_ID顺序从0开始排列GPU设备
# 2、os.environ["CUDA_VISIBLE_DEVICES"]="0"#设置当前使用的GPU设备仅为0号设备 设备名称为‘/gpu:0'
# 3、os.environ["CUDA_VISIBLE_DEVICES"]="1"#设置当前使用的GPU设备仅为1号设备 设备名称为'/gpu:0'
# 4、os.environ["CUDA_VISIBLE_DEVICES"]="0,1"#设置当前的GPU设备为0，1号两个设备，名称依次为’/gpu:0'，'/gpu:1'。表示优先使用0号设备，然后使用1号设备。
# 5、os.environ["CUDA_VISIBLE_DEVICES"="-1"#设置当前的设备使用CPU
# """

In [3]:
#@title Define GANSpace functions
#### 定义 GANSpace 函数

# from ipywidgets import fixed
#ipywidgets: 交互式输入用到的包

# Taken from https://github.com/alexanderkuk/log-progress
# 显示进程
# def log_progress(sequence, every=1, size=None, name='Items'):
#     from ipywidgets import IntProgress, HTML, VBox
#     from IPython.display import display

#     is_iterator = False
#     if size is None:
#         try:
#             size = len(sequence)
#         except TypeError:
#             is_iterator = True
#     if size is not None:
#         if every is None:
#             if size <= 200:
#                 every = 1
#             else:
#                 every = int(size / 200)     # every 0.5%
#     else:
#         assert every is not None, 'sequence is iterator, set every'

#     if is_iterator:
#         progress = IntProgress(min=0, max=1, value=1)
#         progress.bar_style = 'info'
#     else:
#         progress = IntProgress(min=0, max=size, value=0)
#     label = HTML()
#     box = VBox(children=[label, progress])
#     display(box)

#     index = 0
#     try:
#         for index, record in enumerate(sequence, 1):
#             if index == 1 or index % every == 0:
#                 if is_iterator:
#                     label.value = '{name}: {index} / ?'.format(
#                         name=name,
#                         index=index
#                     )
#                 else:
#                     progress.value = index
#                     label.value = u'{name}: {index} / {size}'.format(
#                         name=name,
#                         index=index,
#                         size=size
#                     )
#             yield record
#     except:
#         progress.bar_style = 'danger'
#         raise
#     else:
#         progress.bar_style = 'success'
#         progress.value = index
#         label.value = "{name}: {index}".format(
#             name=name,
#             index=str(index or '?')
#         )

# 保存、展示、返回图片
def display_sample_pytorch(seed=None, truncation=0.5, directions=None, distances=None, scale=1, start=0, end=14, w=None, disp=True, save=None):
    # blockPrint()
    model.truncation = truncation
    
    if w is None:
        w = model.sample_latent(1, seed=seed).detach().cpu().numpy() # samplel:样本；即输入网络的图片，在人脸属性编辑是为1，在人脸融合时为2
        w = [w]*model.get_max_latents() # one per layer
    else:
        w = [np.expand_dims(x, 0) for x in w]
    
    if directions != None and distances != None:
        for l in range(start, end): # 从start层到end层
          for i in range(len(directions)):
            w[l] = w[l] + directions[i] * distances[i] * scale
                        #  属性方向        变化距离
            
    
    torch.cuda.empty_cache() # 删除一些不需要的变量代
    #save image and display
    out = model.sample_np(w)
    
    
    final_im = Image.fromarray((out * 255).astype(np.uint8)).resize((500,500),Image.LANCZOS) # 实现array到image的转换
    
    
    if save is not None:# 保存图片
      if disp == False:
        print(save)
      final_im.save(f'out/{seed}_{save:05}.png')
    if disp: # 显示图片
      display(final_im)
    
    return final_im 

In [2]:
#@title Load Model for GANSpace
selected_model = 'character' #@param ["portrait", "character", "model", "lookbook"]

# Load model
from IPython.utils import io
import torch
import PIL # 该软件包提供了基本的图像处理功能，如：改变图像大小，旋转图像，图像格式转换，色场空间转换，图像增强，直方图处理，插值和滤波等等
import numpy as np
import ipywidgets as widgets # 用于jupyter笔记本和ipython内核的交互式html小部件
from PIL import Image
import imageio # 提供了一个易于阅读和 编写广泛的图像数据，包括动画图像、体积 数据和科学格式
from models import get_instrumented_model
from decomposition import get_or_compute
from config import Config # 网络的配置
from skimage import img_as_ubyte# 基于python脚本语言开发的数字图片处理包

# Speed up computation
torch.autograd.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True

# Specify model to use
config = Config(
  model='StyleGAN2',
  layer='style',
  output_class=selected_model,
  components=80,
  use_w=True,
  batch_size=5_000, # style layer quite small
)

inst = get_instrumented_model(config.model, config.output_class,
                              config.layer, torch.device('cuda'), use_w=config.use_w)

path_to_components = get_or_compute(config, inst) 
# 如果需要，返回缓存结果或计算
# 传递现有的 Instrumented Model 实例以重用它

model = inst.model # styleGAN2

comps = np.load(path_to_components) # 加载主成分分析后的成分  npz文件
# print(len(comps)) # 8
# print(len(comps['act_comp'])) # 80
# print(len(comps['act_comp'][0][0])) # 512
# print(comps['act_comp'])
# print(path_to_components) # gancreate-saai-main/cache/components/stylegan2-character_style_ipca_c80_n300000_w.npz


lst = comps.files
# print(lst)

latent_dirs = []
latent_stdevs = []

load_activations = False

for item in lst:
    if load_activations:
      if item == 'act_comp':
        for i in range(comps[item].shape[0]):
          latent_dirs.append(comps[item][i])
      if item == 'act_stdev':
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])
    else:
      if item == 'lat_comp':
        for i in range(comps[item].shape[0]):
          latent_dirs.append(comps[item][i])
      if item == 'lat_stdev':
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])
# print(len(latent_dirs)) # 80: 取前80个主成分的方向
# print(len(latent_dirs[0])) # 1
# print(len(latent_dirs[0][0])) # 512：latent code的维度即主成分的方向

# print(latent_stdevs) # [ ,80]

def load_model(output_class):
    global config
    global inst
    global model
    config = Config(
    model='StyleGAN2',
    layer='style',
    output_class=output_class,
    components=80,
    use_w=True,
    batch_size=5_000, # style layer quite small
  )

    inst = get_instrumented_model(config.model, config.output_class,
                                  config.layer, torch.device('cuda'), use_w=config.use_w)

    path_to_components = get_or_compute(config, inst)

    model = inst.model

    comps = np.load(path_to_components)
    lst = comps.files
    latent_dirs = []
    latent_stdevs = []

    load_activations = False

    for item in lst:
        if load_activations:
          if item == 'act_comp':
            for i in range(comps[item].shape[0]):
              latent_dirs.append(comps[item][i])
          if item == 'act_stdev':
            for i in range(comps[item].shape[0]):
              latent_stdevs.append(comps[item][i])
        else:
          if item == 'lat_comp':
            for i in range(comps[item].shape[0]):
              latent_dirs.append(comps[item][i])
          if item == 'lat_stdev':
        
            for i in range(comps[item].shape[0]):
              latent_stdevs.append(comps[item][i])

StyleGAN2: Optimized CUDA op FusedLeakyReLU not available, using native PyTorch fallback.
StyleGAN2: Optimized CUDA op UpFirDn2d not available, using native PyTorch fallback.


: 

: 

In [1]:
import torch


### GANSpace UI

In [ ]:
#@title Character Gradio UI
import gradio as gr # 一个开源的构建自动化工具
import numpy as np
import hashlib

def generate_image(seed, truncation,
                  monster, female, skimpy, light, bodysuit, bulky, human_head,
                  start_layer, end_layer):

    scale = 1
    seed = int(hashlib.sha256(seed.encode('utf-8')).hexdigest(), 16) % 10**8

    params = {'monster': monster,
          'female': female,
          'skimpy': skimpy,
          'light': light,
          'bodysuit': bodysuit,
          'bulky': bulky,
          'human_head': human_head}

    param_indexes = {'monster': 0,
              'female': 1,
              'skimpy': 2,
              'light': 4,
              'bodysuit': 5,
              'bulky': 6,
              'human_head': 8}

    directions = []
    distances = []
    for k, v in params.items():
        directions.append(latent_dirs[param_indexes[k]])  # 对应主成分的方向
        distances.append(v) # 对应主成分改变的距离
#     print(distances)

    style = {'description_width': 'initial'}
    return display_sample_pytorch(int(seed), truncation, directions, distances, scale, int(start_layer), int(end_layer), disp=False)

truncation = gr.inputs.Slider(minimum=0, maximum=1, default=0.5, label="Truncation")
start_layer = gr.inputs.Number(default=0, label="Start Layer")
end_layer = gr.inputs.Number(default=14, label="End Layer")
seed = gr.inputs.Textbox(default='0')

slider_max_val = 20
slider_min_val = -20
slider_step = 1

monster = gr.inputs.Slider(label="Monster", minimum=slider_min_val, maximum=slider_max_val, default=0)
female = gr.inputs.Slider(label="Female", minimum=slider_min_val, maximum=slider_max_val, default=0)
skimpy = gr.inputs.Slider(label="Skimpy", minimum=slider_min_val, maximum=slider_max_val, default=0)
light = gr.inputs.Slider(label="Light", minimum=slider_min_val, maximum=slider_max_val, default=0)
bodysuit = gr.inputs.Slider(label="Bodysuit", minimum=slider_min_val, maximum=slider_max_val, default=0)
bulky = gr.inputs.Slider(label="Bulky", minimum=slider_min_val, maximum=slider_max_val, default=0)
human_head = gr.inputs.Slider(label="Human Head", minimum=slider_min_val, maximum=slider_max_val, default=0)


scale = 1

inputs = [seed, truncation, monster, female, skimpy, light, bodysuit, bulky, human_head, start_layer, end_layer]

gr.Interface(generate_image, inputs, "image", live=True, title="CharacterGAN").launch(debug=True,share=True)

c:\Users\Admin\.conda\envs\deploy\lib\site-packages\gradio\inputs.py:88: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
c:\Users\Admin\.conda\envs\deploy\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
c:\Users\Admin\.conda\envs\deploy\lib\site-packages\gradio\inputs.py:58: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
c:\Users\Admin\.conda\envs\deploy\lib\site-packages\gradio\inputs.py:26: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
c:\Users\Admin\.conda\envs\deploy\lib\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, a

Running on local URL:  http://127.0.0.1:7860/
Running on public URL: https://17968.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces
